## **Part 3: Estimation**

In [1]:
# Plotting
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
pio.renderers.default = "notebook+pdf"
pio.templates.default = "plotly_dark"

# Utilities
import numpy as np
from tqdm import tqdm

### **Task 12**

In [2]:
num_women = 1000

# double check if this is the right Q matrix
Q = np.array([[0, 0.0025, 0.00125, 0, 0.001],
              [0, 0, 0, 0.002, 0.005],
              [0, 0, 0, 0.003, 0.005],
              [0, 0, 0, 0, 0.009],
              [0, 0, 0, 0, 0]])

for i in range(Q.shape[0]):
    Q[i,i] = -np.sum(Q[i,:])

print(Q)

[[-0.00475  0.0025   0.00125  0.       0.001  ]
 [ 0.      -0.007    0.       0.002    0.005  ]
 [ 0.       0.      -0.008    0.003    0.005  ]
 [ 0.       0.       0.      -0.009    0.009  ]
 [ 0.       0.       0.       0.      -0.     ]]


In [3]:
def changeState(state, currentTime, endTime, Q):
    Qdiag = np.diag(Q)
    if state == 4:
        return endTime, 4
    qii = Qdiag[state]
    if qii == 0:
        return endTime, state
    time = np.random.exponential(-(1/qii))
    if currentTime + time > endTime:
        return endTime, state
    # Determine the next state
    ps = np.copy(Q[state, :])
    ps[state] = 0
    ps /= -qii
    ps[state] = 1 - sum(ps)
    nextState = np.random.choice(range(5), p=ps)
    return time, nextState

def simulateWoman(startingState, endTime, Q):
    state = startingState
    time = 0
    times = [0]
    states = [state]
    while state != 4 and time < endTime:
        dt, state = changeState(state, time, endTime, Q)
        time += dt
        times.append(time)
        states.append(state)
    return np.array(times), np.array(states)

def simulateWomen(n, startingStates, endTime, Q):
    women = []
    for i in range(n):
        times, states = simulateWoman(startingStates[i], endTime, Q)
        women.append((times, states))
    return women

endTime = np.inf
startingStates = np.zeros(num_women, dtype=int)
women = simulateWomen(num_women, startingStates, endTime, Q)


In [4]:
max_time = 0
for i in range(num_women):
    temp = women[i][0][-1]

    if temp > max_time:
        max_time = temp

time_steps = np.ceil(max_time/48)

Y = np.zeros((num_women, int(time_steps+1)), dtype=int)

for i in range(num_women):
    for j in range(int(time_steps+1)):
        true_indices = np.where([women[i][0] <= 48*j])[1]
        if true_indices.size == 0:
            Y[i,j] = 0
        else:
            Y[i,j] = women[i][1][true_indices.max()]


### **Task 13**

In [134]:
def est_Q(Q_0, Y):
    Q = Q_0
    err = 1

    time_steps = Y.shape[1]
    num_women = Y.shape[0]

    k = 0

    while np.abs(err) > 1e-3:
        k += 1
        print(f"Iteration: {k}")

        N = np.zeros((5,5))
        S = np.zeros((5))

        Q_old = np.copy(Q)

        for i in tqdm(range(num_women)):
            for j in range(1, int(time_steps)):
                while True:
                    women = simulateWomen(1, [Y[i,j-1]], 48, Q_old)

                    Y_new = women[0][1][-1]

                    if Y_new == Y[i,j]:
                        N[Y[i,j-1],Y_new] += 1
                        S[Y[i,j]] += women[0][0][-1]
                        break

        Q = np.zeros((5,5))

        for i in range(5):
            for j in range(5):
                if i != j:
                    Q[i,j] = N[i,j]/S[i]

            Q[i,i] = -np.sum(Q[i,:])

        err = np.linalg.norm(Q_old - Q)

    return Q

#Q_0 = np.copy(Q)

def generate_matrix():
    # Generate a 5x5 matrix with random positive elements
    matrix = np.random.rand(5, 5)

    # Make the lower triangular part zero
    matrix = np.triu(matrix)

    # Make the diagonal elements the negative sum of the remaining row elements
    np.fill_diagonal(matrix, -matrix.sum(axis=1) + np.diag(matrix))

    return matrix

Q_0 = generate_matrix()

print(Q_0)

#est_Q(Q_0, Y)

[[-1.90358277  0.15038147  0.51317055  0.39472257  0.84530819]
 [ 0.         -1.63752038  0.36215674  0.72069198  0.55467166]
 [ 0.          0.         -0.87123149  0.08188046  0.78935103]
 [ 0.          0.          0.         -0.65832623  0.65832623]
 [ 0.          0.          0.          0.          0.        ]]


In [133]:
Q_0 = generate_matrix()
print("Random matrix")
print(Q_0)




Random matrix
[[-0.94711707  0.22797415  0.1389257   0.05014876  0.53006846]
 [ 0.         -1.24082709  0.59678155  0.45706722  0.18697832]
 [ 0.          0.         -1.17052026  0.61214284  0.55837742]
 [ 0.          0.          0.         -0.22933681  0.22933681]
 [ 0.          0.          0.          0.          0.        ]]
